In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [3]:
frame = DataFrame({"key":list("bbacaab"), "data1":range(7)})
frame

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
frame2 = DataFrame({"key":list("abd"), "data2":range(3)})
frame2

,data2,key
0,0,a
1,1,b
2,2,d


`merge` does the same as a relational database "join".  In the following basic form, it automatically decides to merge on `key` as they column name is present in both `DataFrame` objects.

In [6]:
pd.merge(frame, frame2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [8]:
frame3 = DataFrame({"lkey":list("bbacaab"), "data1":range(7)})
frame4 = DataFrame({"rkey":list("abd"), "data2":range(3)})
pd.merge(frame3, frame4, left_on="lkey", right_on="rkey")

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


This, in relational database language, is an "inner" join, so for example the data when `rkey=='d'` is dropped.  You can also perform `outer` and `left` and `right` joins.

In [9]:
pd.merge(frame3, frame4, left_on="lkey", right_on="rkey", how="outer")

,data1,lkey,data2,rkey
0,0.0,b,1.0,b
1,1.0,b,1.0,b
2,6.0,b,1.0,b
3,2.0,a,0.0,a
4,4.0,a,0.0,a
5,5.0,a,0.0,a
6,3.0,c,NaN,NaN
7,NaN,NaN,2.0,d


In [11]:
frame5 = DataFrame({"key":list("bbacab"), "data1":range(6)})
frame6 = DataFrame({"key":list("ababd"), "data2":range(5)})
pd.merge(frame5, frame6, on="key", how="left")

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


The way this works is to _always_ form the product of the two data sources.  The `how` parameter just determines which, if any, entries are then dropped.

## Merging using indexes ##

In [13]:
# On one side we want to use the index
frame1 = DataFrame({"key":list("abaacb"), "value":range(6)})
frame2 = DataFrame({"group":[1.2, -3.2]}, index=list("ab"))
pd.merge(frame1, frame2, left_on="key", right_index=True)

,key,value,group
0,a,0,1.2
2,a,2,1.2
3,a,3,1.2
1,b,1,-3.2
5,b,5,-3.2


In [14]:
pd.merge(frame1, frame2, left_on="key", right_index=True, how="outer")

,key,value,group
0,a,0,1.2
2,a,2,1.2
3,a,3,1.2
1,b,1,-3.2
5,b,5,-3.2
4,c,4,NaN


In [19]:
frame3 = DataFrame({"key1":["Leeds", "Leeds", "Leeds", "Bradford", "Bradford"],
                   "key2":[2015,2016,2017,2015,2016],
                   "data":range(5)})
frame4 = DataFrame(np.random.random((6,2)),
                  index=[["Bradford", "Bradford", "Leeds", "Leeds", "Leeds", "Leeds"],
                        [2015,2016,2015,2015,2016,2016]],
                  columns=["event1", "event2"])
frame4

event1    event2
Bradford 2015  0.180138  0.889123
         2016  0.032400  0.023167
Leeds    2015  0.014729  0.522196
         2015  0.772776  0.605179
         2016  0.228784  0.719213
         2016  0.025381  0.637467

In [20]:
pd.merge(frame3, frame4, left_on=["key1", "key2"], right_index=True)

,data,key1,key2,event1,event2
0,0,Leeds,2015,0.014729,0.522196
0,0,Leeds,2015,0.772776,0.605179
1,1,Leeds,2016,0.228784,0.719213
1,1,Leeds,2016,0.025381,0.637467
3,3,Bradford,2015,0.180138,0.889123
4,4,Bradford,2016,0.032400,0.023167


In [22]:
pd.merge(frame3, frame4, left_on=["key1", "key2"], right_index=True, how="outer")

,data,key1,key2,event1,event2
0,0,Leeds,2015,0.014729,0.522196
0,0,Leeds,2015,0.772776,0.605179
1,1,Leeds,2016,0.228784,0.719213
1,1,Leeds,2016,0.025381,0.637467
2,2,Leeds,2017,NaN,NaN
3,3,Bradford,2015,0.180138,0.889123
4,4,Bradford,2016,0.032400,0.023167


# Concatenating #

In [25]:
array = np.random.random((2,3))
array

array([[ 0.17035809,  0.93713245,  0.50986323],
       [ 0.08219274,  0.02367718,  0.55031617]])

In [26]:
np.concatenate([array, array], axis=1)

array([[ 0.17035809,  0.93713245,  0.50986323,  0.17035809,  0.93713245,
         0.50986323],
       [ 0.08219274,  0.02367718,  0.55031617,  0.08219274,  0.02367718,
         0.55031617]])

In [27]:
np.concatenate([array, array], axis=0)

array([[ 0.17035809,  0.93713245,  0.50986323],
       [ 0.08219274,  0.02367718,  0.55031617],
       [ 0.17035809,  0.93713245,  0.50986323],
       [ 0.08219274,  0.02367718,  0.55031617]])

Pandas can also do this, taking account of indexes.

The following is the easiest situation:

In [28]:
ser1 = Series([0,1], index=list("ab"))
ser2 = Series([2,3,4], index=list("cde"))
ser3 = Series([5,6], index=list("fg"))
pd.concat([ser1,ser2,ser3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [30]:
pd.concat([ser1,ser2,ser3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [32]:
ser4 = pd.concat([ser1*5,ser3])
ser4

a    0
b    5
f    5
g    6
dtype: int64

In [33]:
pd.concat([ser1,ser4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [34]:
pd.concat([ser1,ser4], axis=1, join="inner")

,0,1
a,0,0
b,1,5


In [36]:
pd.concat([ser1,ser4], axis=1, join_axes=[list("acbe")])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


## Creating a hierarchical index ##

In [38]:
result = pd.concat([ser1,ser1,ser3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [39]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


## Creating DataFrames ##

In [40]:
pd.concat([ser1,ser2,ser3], axis=1, keys=["one","two","three"])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0
